In [ ]:
pip install twython

In [22]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os 
dir_name= os.getcwd()+'/data/'

In [2]:
import tweepy

consumer_key = "" 
consumer_secret = "" 
access_token = "" 
access_token_secret = ""


# authorization of consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)




In [11]:
import pandas as pd
import os


dir_name= os.getcwd()+'/data/'

# get the full path of the data folder to get the list of tickers in different stock market
nasdaq_files=os.path.join(dir_name,'stock_ticker_datasets/nasdaq.csv')
nyse_files=os.path.join(dir_name,'stock_ticker_datasets/nyse.csv')
hkex_files=os.path.join(dir_name,'stock_ticker_datasets/hkex_in.txt')

# read in the stock ticker file
nasdaq=pd.read_csv(nasdaq_files) 
nyse=pd.read_csv(nyse_files)
hkex=pd.read_csv(hkex_files, delimiter = '\t')


hkex.columns = ['Symbol','Description']
hkex.to_csv(dir_name+'stock_ticker_datasets/hkex_in.csv',index=False)

# get the stock symbol in each stock market
nasdaq['Symbol']=nasdaq['Symbol'].astype(str)
nasdaq_input=nasdaq['Symbol']


nyse['Symbol']=nyse['Symbol'].astype(str)
nyse_input=nyse['Symbol']

hkex['Symbol']=hkex['Symbol'].astype(str)
hkex_input=hkex['Symbol']



#chunk row size
n = 100  


#split datasets into smaller sets to prevent blocked from the API
nasdaq_df = [nasdaq_input[i:i+n] for i in range(0,nasdaq_input.shape[0],n)]
nyse_df = [nyse_input[i:i+n] for i in range(0,nyse_input.shape[0],n)]
hkex_df = [hkex_input[i:i+n] for i in range(0,hkex_input.shape[0],n)]


nasdaq.set_index("Symbol" , inplace=True)
nyse.set_index("Symbol" , inplace=True)
hkex.set_index("Symbol" , inplace=True)




      Symbol                      Description
0          1       Cheung Kong (Holdings) Ltd
1          2                 CLP Holdings Ltd
2          3  Hong Kong and China Gas Co. Ltd
3          4             Wharf (Holdings) Ltd
4          5                HSBC Holdings plc
...      ...                              ...
1960    9170                              NaN
1961    9834                              NaN
1962    9836                              NaN
1963    9846                              NaN
1964    9847                              NaN

[1965 rows x 2 columns]


In [7]:
import datetime, time
import csv

# get data from the company's twitter timeline
# name: twitter screen name of the ticker
# number_of tweets: limit the number of tweets collected
# day: limit the number of days of the tweets

def returnTimelinetweets(name,number_of_tweets,day):
    # get data from the company's twitter timeline
    alltweets = []
    user=api.get_user(name)
    userid=user.id_str
    # access the timeline tweets through the API
    timeline = api.user_timeline(user_id=userid,count=number_of_tweets)
    alltweets.extend(timeline)
    outtweets=[]
    for tweet in alltweets:
        if(datetime.datetime.now()-tweet.created_at).days<=day:
            outtweets=[tweet.created_at,tweet.text]

    path=os.path.join(dir_name,'data-tweets/')
    with open(path+'data-'+name+'-tweets.csv','a') as f:
        writer= csv.writer(f)
        writer.writerows(outtweets)
            
# get data from  the cashtag
# name: twitter screen name of the ticker
# day: limit the number of days of the tweets

def get_tagtweets_us(name,day):
    allhashtag=[]
    print(name)
    # get the hasgtag/cashtag tweets
    hashtags=tweepy.Cursor(api.search,q=name,lang='en',tweet_mode='extended').items(200)
    outttags=[] 
    # get the csv file of each ticker that contains the tweets
    path=os.path.join(dir_name,'data-tweets/us-tweets/'+'data-'+name+'-tweets.csv')
    print(path)
    try:
        with open(path,'a') as f:
            writer = csv.writer(f)
            for status in hashtags:
                if(datetime.datetime.now()-status.created_at).days<=day:
                    tweet_text = status.full_text
                    # only collect the year, month and day
                    dates=str(status.created_at)[:10]
                    print(dates)
                    print(tweet_text)
                    writer.writerow([dates,tweet_text])
        
            time.sleep(1)
    
    except Exception as e:
        print(e)
        pass
    
# get data from  the cashtag from hkex
# name: twitter screen name of the ticker
# day: limit the number of days of the tweets

def get_tagtweets_hk(name,day):
    
    name=name+'.HK'
    print(name)
    hashtags=tweepy.Cursor(api.search,q=name,lang='en',tweet_mode='extended').items(300)
    print(hashtags)
    path=os.path.join(dir_name,'data-tweets/hk-tweets/'+'data-'+name+'-tweets.csv')
    print(path)
    try:
        with open(path,'a') as f:
            
            writer = csv.writer(f)
            for status in hashtags:
                if(datetime.datetime.now()-status.created_at).days<=day:
                    tweet_text = status.full_text
                    dates=str(status.created_at)[:10]
                    print(dates)
                    print(tweet_text)
                    writer.writerow([dates,tweet_text])

        time.sleep(1)       
    
    except Exception as e:
        print(e)
        pass

 


In [5]:

import tweepy, datetime, time
import csv
import sys


# real time tweets collection using tweepy stream API
days=1
class MyStreamListener(tweepy.StreamListener):
    
    def __init__(self, output_file=sys.stdout,input_name=sys.stdout):
        super(MyStreamListener,self).__init__()
        self.output_file = output_file
        self.max_tweets=200
        self.tweet_count=0
        self.input_name=input_name
        
        
    def on_status(self, status):
        if(self.tweet_count==self.max_tweets):
            return False
        elif(datetime.datetime.now()-status.created_at).days<1:
            tweet_text = status.text
            print(tweet_text)
            writer = csv.writer(self.output_file)
            
            writer.writerow([status.created_at,status.extended_tweet['full_text']])
            self.tweet_count+=1
            
        
    def on_error(self, status_code):
        print(status_code)
        if status_code == 420:
            #returning False in on_error disconnects the stream
            
            return False
        
    def on_timeout(self):
        sys.stderr.write("Timeout, sleeping for 60 seconds...\n")
        time.sleep(60)
        return 
            


# for firm in list:

#         path=os.path.join(dir_name,'data-tweets/')
#         with open(path+'data-'+name+'-tweets.csv','a') as f:
#         writer = csv.writer(f)
#         myStreamListener = MyStreamListener(output_file=f,input_name=firm)
#         myStream = tweepy.Stream(auth = api.auth, tweet_mode='extended',listener=myStreamListener,languages = ["en"])
#         myStream.filter(track=firm)


In [ ]:
# update function to update the tweets using the get_tagtweets function for all stock market
def update_tweets_all_us(day):
    for list in nasdaq_df:
        for name in list:
            name='$'+name
            get_tagtweets(name,day)
    for list in nyse_df:
        for name in list:
            name='$'+name
            get_tagtweets(name,day)
            

def update_tweets_all_hk(day):
    for list in hkex_df:
        for name in list:
            name=name.zfill(4)
            get_tagtweets_hk(name,day)
            
update_tweets_all_hk(1)

0001.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0001.HK-tweets.csv
0002.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0002.HK-tweets.csv
0003.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0003.HK-tweets.csv
0004.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0004.HK-tweets.csv
0005.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0005.HK-tweets.csv
0006.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0006.HK-tweets.csv
0007.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0007.HK-tweets.csv
0008.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0008.HK-tweets.csv
0009.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0009.HK-tweets.csv
0010.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0010.HK-tweets.csv
0011.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0011.HK-tweets.csv
0012.HK
/u

Rate limit reached. Sleeping for: 379


0159.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0159.HK-tweets.csv
2021-02-16
Paul Torres-Cohen $0159.HK
0160.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0160.HK-tweets.csv
0161.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0161.HK-tweets.csv
0162.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0162.HK-tweets.csv
0163.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0163.HK-tweets.csv
0164.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0164.HK-tweets.csv
0165.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0165.HK-tweets.csv
0166.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0166.HK-tweets.csv
0167.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0167.HK-tweets.csv
0168.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0168.HK-tweets.csv
0169.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-twe

Rate limit reached. Sleeping for: 673


0326.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0326.HK-tweets.csv
0327.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0327.HK-tweets.csv
0328.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0328.HK-tweets.csv
0329.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0329.HK-tweets.csv
0330.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0330.HK-tweets.csv
0331.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0331.HK-tweets.csv
0332.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0332.HK-tweets.csv
0333.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0333.HK-tweets.csv
0334.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0334.HK-tweets.csv
0335.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0335.HK-tweets.csv
0336.HK
/userhome/30/xwu/anaconda3/fyp/data/data-tweets/hk-tweets/data-0336.HK-tweets.csv
0337.HK
/u

In [ ]:
# return a pandas dataframe contains today's tweets
# name: name of the ticker
def get_today_tweets(name,day):
    text=[]
    date=[]
    tweets=pd.DataFrame()
    
    hashtags=tweepy.Cursor(api.search,q=name,lang='en',tweet_mode='extended').items(300)
    outttags=[] 
    path=os.path.join(dir_name,'data-tweets/')
    for status in hashtags:
        if(datetime.datetime.now()-status.created_at).days<=day:
            tweet_text = status.full_text.encode("utf-8")
            dates=str(status.created_at)[:10]
            text.append(tweet_text)
            dates.append(dates)
    tweets['date']=dates
    
    ticker=[name for i in range(len(dates))]
    tweets['ticker']=ticker
    tweets['text']=text
    
    return tweets
    
    
    
    

    

In [ ]:
# merging all the tweets csv file in  nasdaq
# name: name of the ticker
def merge_nasdaq_no_label_tweets(name):
    df = pd.DataFrame()
    for nas in nyse_df:
         for name in nas:
            try:
                path=os.path.join(dir_name,'data-tweets/'+'data-$'+name+'-tweets.csv')
                df_tweets = pd.read_csv(path ,names=['dates','tweets'],index_col='dates')
                sector=nyse.loc[name]['Sector']
                ticker=[name for i in range(len(df_tweets))]
                Sector=[sector for i in range(len(df_tweets))]
                df_tweets['ticker']=ticker
                df_tweets['sector']=Sector
                df=df.append(df_tweets)


            except:
                print('no such file exist')
                pass
            df_path=os.path.join(dir_name,'train-data/nasdaq/nasdaq_no_labelled_tweets.csv')
            df.to_csv(df_path)

In [ ]:
# merging all the tweets csv file in  nyse
# name: name of the ticker
def merge_nyse_no_label_tweets(name):
    df = pd.DataFrame()
    for nas in nyse_df:
         for name in nas:
            try:
                path=os.path.join(dir_name,'data-tweets/'+'data-$'+name+'-tweets.csv')
                df_tweets = pd.read_csv(path ,names=['dates','tweets'],index_col='dates')
                sector=nyse.loc[name]['Sector']
                ticker=[name for i in range(len(df_tweets))]
                Sector=[sector for i in range(len(df_tweets))]
                df_tweets['ticker']=ticker
                df_tweets['sector']=Sector
                df=df.append(df_tweets)


            except:
                print('no such file exist')
                pass
            df_path=os.path.join(dir_name,'train-data/nyse/nyse_no_labelled_tweets.csv')
            df.to_csv(df_path)